# Reading Avro Blobs Into Parquet Data Sets

### Dependency Importing and Environment Variable Retrieval

In [46]:
import os
import string
import json
import pandas as pd
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import udf
from pyspark.sql.types import TimestampType, StringType
from pyspark.storagelevel import StorageLevel
from azure.storage.table import TableService

#### Read Environment Variables

In [47]:
#For development purposes only until ENV Variables get set
from pathlib import Path
env_config_file_location = (str(Path.home())+"/notebooks/NotebookEnvironmentVariablesConfig.JSON")
f = open(env_config_file_location)
env_variables = json.load(f)["DataIngestion"]

STORAGE_ACCOUNT_SUFFIX = 'core.windows.net'
STORAGE_ACCOUNT_NAME = env_variables["STORAGE_ACCOUNT_NAME"]
STORAGE_ACCOUNT_KEY = env_variables["STORAGE_ACCOUNT_KEY"]
TELEMETRY_CONTAINER_NAME = env_variables["TELEMETRY_CONTAINER_NAME"]
LOG_TABLE_NAME = env_variables["LOG_TABLE_NAME"]
DATA_ROOT = env_variables["DATA_ROOT_FOLDER"]

### Setting up Drop Folder

In [48]:
data_dir = DATA_ROOT + '/data'

#TODO: Convert data_dir into env variable
% rm -rf $data_dir
% mkdir $data_dir $data_dir/logs

### Retrieving telemetry data (as spark dataframe)

In [49]:
wasbTelemetryUrl = "wasb://{0}@{1}.blob.{2}/*/*/*/*/*/*/*".format(TELEMETRY_CONTAINER_NAME, 
                                                                  STORAGE_ACCOUNT_NAME, 
                                                                  STORAGE_ACCOUNT_SUFFIX)

sc = SparkSession.builder.getOrCreate()
hc = sc._jsc.hadoopConfiguration()
hc.set("avro.mapred.ignore.inputs.without.extension", "false")
if STORAGE_ACCOUNT_KEY:
     hc.set("fs.azure.account.key.{}.blob.core.windows.net".format(STORAGE_ACCOUNT_NAME), STORAGE_ACCOUNT_KEY)
hc.set("fs.azure.account.key.{}.blob.core.windows.net"
    .format(STORAGE_ACCOUNT_NAME), STORAGE_ACCOUNT_KEY)
sql = SQLContext.getOrCreate(sc)
avroblob = sql.read.format("com.databricks.spark.avro").load(wasbTelemetryUrl)
avroblob.show()

+--------------------+----------+--------------------+--------------------+
|     EnqueuedTimeUtc|Properties|    SystemProperties|                Body|
+--------------------+----------+--------------------+--------------------+
|2018-06-26T00:45:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-26T00:45:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-26T00:45:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-26T00:45:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-26T00:45:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-26T00:45:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-26T00:45:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-26T00:45:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-26T00:45:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-26T00:45:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-26T

### Convert byteformatted "body" of raw blob data into JSON, then explode result into new Pyspark DataFrame

In [50]:
#Convert byteformat to string format in pyspark dataframe
from json import loads as Loads
column = avroblob['Body']
string_udf = udf(lambda x: x.decode("utf-8"))
avroblob=avroblob.withColumn("BodyString", string_udf(column))
avroblob.printSchema()

#Convert "body" into new DataFrame
telemetry_df = sql.read.json(avroblob.select("BodyString").rdd.map(lambda r: r.BodyString))
telemetry_df.show()

root
 |-- EnqueuedTimeUtc: string (nullable = true)
 |-- Properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- SystemProperties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- Body: binary (nullable = true)
 |-- BodyString: string (nullable = true)

+----------------+-------------------+-----------+--------+-------+-------------+-----------+--------------------+---------+
|ambient_pressure|ambient_temperature|  machineID|pressure|  speed|speed_desired|temperature|           timestamp|vibration|
+----------------+-------------------+-----------+--------+-------+-------------+-----------+--------------------+---------+
|          100.96|              19.95|Machine-003| 1448.84| 1078.2|         1000|     135.51|2018-06-26T00:45:...|     null|
|           101.0|              20.06|Machine-004|  868.15|1030.75|         1000|     151.18|2018-06-26T00:45:...|     null|
|          100

In [51]:
#columns to retain: timestamp, ambient_pressure, ambient_temperature machineID, pressure, speed, 
#                   speed_desired, temperature
subsetted_df = telemetry_df.select(["timestamp", "ambient_pressure","ambient_temperature","machineID","pressure","speed","speed_desired","temperature"])


In [52]:
#modify timestamp format
import datetime
e = '%Y-%m-%dT%H:%M:%S.%f'
timestamp_udf = udf(lambda date: datetime.datetime.strptime(date, e), TimestampType())
reformatted_time_df = subsetted_df.withColumn("timestamp", timestamp_udf(subsetted_df["timestamp"]))

reformatted_time_df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- ambient_pressure: double (nullable = true)
 |-- ambient_temperature: double (nullable = true)
 |-- machineID: string (nullable = true)
 |-- pressure: double (nullable = true)
 |-- speed: double (nullable = true)
 |-- speed_desired: long (nullable = true)
 |-- temperature: double (nullable = true)



### Write dataframe to Parquet in system storage

In [53]:
reformatted_time_df.write.parquet(data_dir+"/telemetry", mode="overwrite")

## Get Logs

In [54]:
#table retrieval
table_service = TableService(account_name=STORAGE_ACCOUNT_NAME, account_key=STORAGE_ACCOUNT_KEY)
tblob = table_service.query_entities(LOG_TABLE_NAME)

### Process log table data into Pandas DataFrame

In [55]:
attributes = list()
for row in tblob:
    if (len(attributes) == 0):
        for attribute in row:
            attributes.append(attribute)
    break
log_df = pd.DataFrame(columns=attributes)
for row in tblob:
    if (row["Level"] != "DEBUG"):
        row_dict = {}    
        for attribute in row:
            if (attribute != "Timestamp"):
                row_dict[attribute] = row[attribute]
            else:
                newtime = row[attribute].replace(tzinfo=None)
                timeitem = pd.Timestamp(newtime, tz=None)
                row_dict[attribute] = timeitem
        log_df = log_df.append(row_dict, ignore_index=True)
log_df.head()

,etag,Message,Code,Timestamp,_Driver,PartitionKey,RowKey,Level
0,"W/""datetime'2018-06-27T01%3A48%3A27.276968Z'""",Simulation started.,NaN,2018-06-27 01:48:27.276968,2155f98b-1e3b-495a-b5b9-451cb87c0f1d,Machine-000,2ee434dc37e846919721511323c93153,INFO
1,"W/""datetime'2018-06-27T01%3A58%3A39.0637119Z'""",Simulation started.,NaN,2018-06-27 01:58:39.063711,1ab7eae1-fb35-419c-9485-b644280a2a23,Machine-000,4ca50e21e49a4fc7883bfbb557482e98,INFO
2,"W/""datetime'2018-06-27T03%3A28%3A41.3005356Z'""",Simulation started.,NaN,2018-06-27 03:28:41.300535,3fe41f9e-9ba0-4b9d-b072-fc4364f256f3,Machine-000,5768022591c44433893532a1fc0ea075,INFO
3,"W/""datetime'2018-06-27T04%3A02%3A15.5696863Z'""",Simulation started.,NaN,2018-06-27 04:02:15.569686,7d5806ca-5e02-498a-a599-fd296b70bac6,Machine-000,585d6262305240b5987eb235845e9dad,INFO
4,"W/""datetime'2018-06-27T03%3A50%3A16.5535432Z'""",Simulation started.,NaN,2018-06-27 03:50:16.553543,dbaa3b25-8cc1-4c90-b509-f751277a26d1,Machine-000,5b692f03bc214d128905564f1fd9f2a2,INFO


### Number of Run-To-Failure Sequences

In [56]:
log_df = log_df[log_df.Message != 'failure']

In [61]:
message_counts = log_df['Message'].value_counts()
if ('failure' in message_counts):
    print("Number of Run-to-Failures:", message_counts['failure'])
else:
    raise ValueError('Run to failure count is 0. Do not proceed.')

KeyError: 'Message'

### Select necessary attributes

In [58]:
log_df = log_df[["Timestamp", "Code", "Level", "PartitionKey"]]
log_df.columns = ["timestamp", "code","level","machineID"]
log_df.index = log_df['timestamp']
del log_df['timestamp']
log_df.head()

,code,level,machineID
timestamp,,,
2018-06-27 01:48:27.276968,NaN,INFO,Machine-000
2018-06-27 01:58:39.063711,NaN,INFO,Machine-000
2018-06-27 03:28:41.300535,NaN,INFO,Machine-000
2018-06-27 04:02:15.569686,NaN,INFO,Machine-000
2018-06-27 03:50:16.553543,NaN,INFO,Machine-000


### Write logs to system storage

In [59]:
log_df.to_parquet(data_dir+"/logs/logs.parquet", engine='fastparquet', times='int96')